# Task 2
- Raghad Al-Rasheed
- Fawwaz Alsheikhi

using the E5 model as the embedding model and translated dataset from huggingface

In [1]:
!pip install sentence_transformers

## Downloading the Embedding model

In [2]:
from sentence_transformers import SentenceTransformer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import math
from scipy import spatial


model = SentenceTransformer("intfloat/multilingual-e5-large").to('cuda')

/home/fawaz/Documents/private/Tuwaiq/week1_assesment/env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /home/fawaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Downloading Translated data from english to arabic

In [3]:
!pip3 install datasets
from datasets import load_dataset


ds = load_dataset("Helsinki-NLP/news_commentary", "ar-en",split="train")

In [4]:
import pandas as pd

df = pd.DataFrame(ds['translation'])

In [5]:
df['ar']

0                                  الذهب بعشرة آلاف دولار؟
1        سان فرانسيسكو ـ لم يكن من السهل قط أن ينخرط ال...
2        ولكن من المؤسف أن أسعار الذهب واصلت ارتفاعها م...
3        ومؤخراً شرح لي أحد المستثمرين الناجحين في مجال...
4        لا أحد يستطيع أن ينكر أن توقع ارتفاع أسعار الذ...
                               ...                        
83182    لا شك أن المخاطر التي تحيط بأفريقيا هائلة. إن ...
83183    إن زوما يستشعر الوضع الحرج الذي تعيشه جنوب أفر...
83184    رغم ذلك، وطبقاً لرأي الخبير الاقتصادي السياسي ...
83185    ولكن في الوقت نفسه ما زال جيل زوما الثوري يظهر...
83186    إن ثلاثة من كل عشرة في جنوب أفريقيا شباب دون ا...
Name: ar, Length: 83187, dtype: object

In [6]:
df['ar'][0]

'الذهب بعشرة آلاف دولار؟'

### Extracting the first 10000 rows out of the data

In [7]:
df=df.head(10000)

In [8]:
df['ar'].shape

(10000,)

In [9]:
documents =[doc for doc in df['ar']]

In [10]:
documents[9999]

'إن إنقاذ ولاية ضخمة مثل كاليفورنيا، في حين أنه يضيف إلى الضغوط التي تتعرض لها حكومة أوباما ، فمن شأنه أيضاً أن يكشف الحلقة الضعيفة في نظام الحكم في الولايات المتحدة. إن البلدان التي يطلق عليها دولاً &quot;وحدوية&quot;، مثل بريطانيا أو فرنسا أو الصين أو كينيا، لديها في الأساس مجموعة منفردة من الالتزامات الحكومية: جهاز شرطة وطني واحد، وصاحب عمل واحد لكل معلمي المدارس العامة، ونظام إجمالي واحد لمعاشات التقاعد، إلى آخر ذلك. وعلى النقيض من ذلك سنجد أن الولايات المتحدة تتسم بشكل &quot;غير متماثل&quot; من أشكال الحكم، الأمر الذي يسمح بنشوء العديد من الكيانات الحكومية المتداخلة ـ سبعة آلاف في كاليفورنيا وحدها ـ التي تتعامل مع الديون، وتعين أو تفصل الموظفين، وتفرض الضرائب.'

## Embedding the sentences by rows

In [11]:
embeddings = model.encode(documents)

In [12]:
from sentence_transformers import SentenceTransformer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import math
from scipy import spatial
import scipy

def semantic_search(query, embeddings, documents):
    query_embedding = model.encode(query)

    document_embeddings = embeddings
    scores = [scipy.spatial.distance.cosine(query_embedding, doc) for doc in document_embeddings]
    ls1 = list()
    for i, score in enumerate(scores):
        ls1.append([documents[i],score])

    print(scores.index(min(scores)))
    most_similar_doc = documents[scores.index(min(scores))]
    print("Most similar document", most_similar_doc)
    return ls1

[nltk_data] Downloading package punkt to /home/fawaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
output = semantic_search("ـ لم يكن من السهل قط أن ينخرط المرء في محادثة عقلانية حول قيمة الذهب.",embeddings, documents)

5855
Most similar document بيد أن قصة الأرز الذهبي تشير بوضوح إلى أننا لم نكتسب بعد القدر اللازم من الإرادة والحكمة لتحقيق تلك الغايات.


In [14]:
documents[999]

'حين سافر جيانغ زيمين سلف هيو إلى اليابان منذ عشرة أعوام، كانت العلاقات الثنائية بين البلدين تشهد نوعاً من التدهور: حيث لم تكن الصين سعيدة إزاء رفض الحكومة اليابانية تقديم اعتذاراها للصين بسبب اعتداءاتها الماضية، كما فعلت مع كوريا الجنوبية؛ كما شعرت اليابان بانزعاج بالغ وهي تراقب نهضة الصين ونزوعها إلى المجابهة والتحدي. وتناولت وسائل الإعلام اليابانية الزيارة بقدر عظيم من السلبية، حتى أنها وصِفَت بعد ذلك باعتبارها كارثة دعائية.'

### Extracting top three related sentences

In [15]:
ranked = sorted(output, key=lambda x: x[1])
ranked[:3]

[['بيد أن قصة الأرز الذهبي تشير بوضوح إلى أننا لم نكتسب بعد القدر اللازم من الإرادة والحكمة لتحقيق تلك الغايات.',
  0.1332587679182733],
 ['سان فرانسيسكو ـ لم يكن من السهل قط أن ينخرط المرء في محادثة عقلانية حول قيمة الذهب. ومؤخراً، مع ارتفاع أسعار الذهب بما يزيد على 300% في غضون الأعوام العشرة الماضية، فقد أصبح الأمر أصعب من أي وقت مضى. ففي شهر ديسمبر/كانون الأول الماضي، نشر كل من مارتن فيلدشتاين ونورييل روبيني ـ وهما من أبرز خبراء الاقتصاد ـ مقالاً تحريرياً، حيث شكك كل منهما في مقاله بشجاعة في مشاعر السوق الصاعدة، مشيراً بحكمة إلى المجازفات والمخاطر المحيطة بالذهب.',
  0.14536714553833008],
 ['لا شك أن مثل هذه الاعتبارات قد تكون ذات تأثير طفيف على الأسعار. فما صدق بالنسبة للكيميائيين في الماضي لا يزال صادقاً حتى اليوم: فالتوفيق بين الذهب والمنطق قد يكون أمراً بالغ الصعوبة في كثير من الأحيان.',
  0.14759991012811358]]

In [16]:
df

,ar,en
0,الذهب بعشرة آلاف دولار؟,"$10,000 Gold?"
1,سان فرانسيسكو ـ لم يكن من السهل قط أن ينخرط ال...,"Lately, with gold prices up more than 300% ove..."
2,ولكن من المؤسف أن أسعار الذهب واصلت ارتفاعها م...,"Last December, many gold bugs were arguing tha..."
3,ومؤخراً شرح لي أحد المستثمرين الناجحين في مجال...,One successful gold investor recently explaine...
4,لا أحد يستطيع أن ينكر أن توقع ارتفاع أسعار الذ...,But January 1980 was arguably a “freak peak” d...
...,...,...
9995,بيركلي ـ بينما تستولي إدارة أوباما الجديدة على...,Berkeley – While the new Obama administration ...
9996,كانت كاليفورنيا، وهي الولاية الأكثر ثراءً بين ...,"California’s two most famous industries, Silic..."
9997,ورغم ذلك فإن القوة التكنولوجية والتجارية التي ...,"On May 19, the state’s voters, in a special el..."
9998,وبالنسبة للرئيس باراك أوباما فإن أزمة كاليفورن...,"For President Barack Obama, California’s crisi..."


## using english with arabic to see the semantic search of multilangual model

In [17]:
df['ar']

0                                 الذهب بعشرة آلاف دولار؟
1       سان فرانسيسكو ـ لم يكن من السهل قط أن ينخرط ال...
2       ولكن من المؤسف أن أسعار الذهب واصلت ارتفاعها م...
3       ومؤخراً شرح لي أحد المستثمرين الناجحين في مجال...
4       لا أحد يستطيع أن ينكر أن توقع ارتفاع أسعار الذ...
                              ...                        
9995    بيركلي ـ بينما تستولي إدارة أوباما الجديدة على...
9996    كانت كاليفورنيا، وهي الولاية الأكثر ثراءً بين ...
9997    ورغم ذلك فإن القوة التكنولوجية والتجارية التي ...
9998    وبالنسبة للرئيس باراك أوباما فإن أزمة كاليفورن...
9999    إن إنقاذ ولاية ضخمة مثل كاليفورنيا، في حين أنه...
Name: ar, Length: 10000, dtype: object

In [18]:
df['en']

0                                           $10,000 Gold?
1       Lately, with gold prices up more than 300% ove...
2       Last December, many gold bugs were arguing tha...
3       One successful gold investor recently explaine...
4       But January 1980 was arguably a “freak peak” d...
                              ...                        
9995    Berkeley – While the new Obama administration ...
9996    California’s two most famous industries, Silic...
9997    On May 19, the state’s voters, in a special el...
9998    For President Barack Obama, California’s crisi...
9999    So-called “unitary” nations such as Britain, F...
Name: en, Length: 10000, dtype: object

In [19]:
df_ar = df['ar'].tolist()[:5000]

df_en = df['en'].tolist()[:5000]

combined_list = df_ar + df_en

print(len(combined_list))


10000


In [20]:
embeddings1 = model.encode(combined_list)

In [31]:
from sentence_transformers import SentenceTransformer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import math
from scipy import spatial
import scipy

def semantic_search(query):
    query_embedding = model.encode(query)

    document_embeddings = embeddings1
    scores = [scipy.spatial.distance.cosine(query_embedding, doc) for doc in document_embeddings]
    ls1 = list()
    for i, score in enumerate(scores):
        ls1.append([combined_list[i],score])

    print(scores.index(min(scores)))
    most_similar_doc = combined_list[scores.index(min(scores))]
    ranked = sorted(output, key=lambda x: x[1])
    print("Most similar document", most_similar_doc)
    return ranked[0],ranked[1],ranked[2]

[nltk_data] Downloading package punkt to /home/fawaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
# output = semantic_search("لذهب بعشرة آلاف دولار؟")

0
Most similar document الذهب بعشرة آلاف دولار؟


In [29]:
# ranked = sorted(output, key=lambda x: x[1])
# ranked[:3]

[['الذهب بعشرة آلاف دولار؟', 0.030088365077972412],
 ['$10,000 Gold?', 0.1332303961056691],
 ['ومؤخراً شرح لي أحد المستثمرين الناجحين في مجال الذهب أن أسعار الأسهم كانت هزيلة لأكثر من عشرة أعوام قبل أن يتجاوز مؤشر داو جونز علامة الألف في أوائل ثمانينيات القرن العشرين. ومنذ ذلك الوقت استمر المؤشر في الصعود حتى تجاوز العشرة آلاف نقطة. والآن بعد أن عبر الذهب حاجز الألف السحري، فما الذي يمنعه من الارتفاع إلى عشرة أمثال؟',
  0.14362496137619019]]

In [35]:
import gradio as gr

demo = gr.Interface(fn=semantic_search,inputs = ["text"], outputs=["text", "text", "text"])
if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://e0af48a154548db54b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
